In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# ========== 读取内参 ==========
def load_intrinsics_txt(path):
    vals = np.loadtxt(path)
    fx, fy, cx, cy = vals[:4]
    K = np.array([
        [fx, 0,  cx],
        [0,  fy, cy],
        [0,  0,  1]
    ])
    return K

# ========== 读取外参 ==========
def load_extrinsics_txt(path):
    return np.loadtxt(path)

# ========== 生成 3D BBox ==========
def get_3d_bbox(l, w, h):
    x = [ l/2,  l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2 ]
    y = [ w/2, -w/2, -w/2,  w/2,  w/2, -w/2, -w/2,  w/2 ]
    z = [ 0,    0,    0,    0,   -h,   -h,   -h,   -h ]
    return np.vstack([x, y, z, np.ones(8)])

# ========== 相机坐标系（小三角锥表示） ==========
def get_camera_frustum(T, scale=2.0):
    cam_center = T[:3,3]
    R = T[:3,:3]
    pts = np.array([
        [0,0,0],
        [1, 1, 2],
        [1,-1, 2],
        [-1,-1, 2],
        [-1, 1, 2]
    ]) * scale
    pts_world = (R @ pts.T).T + cam_center
    return cam_center, pts_world

# ========== 可视化 ==========
def plot_scene(T_obj_to_world, T_cam_to_world, box_size):
    l, w, h = box_size
    bbox_obj = get_3d_bbox(l, w, h)
    bbox_world = T_obj_to_world @ bbox_obj

    cam_center, cam_frustum = get_camera_frustum(T_cam_to_world, scale=2.0)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # 画 BBox
    verts = [
        [bbox_world[:3,i] for i in [0,1,2,3]],
        [bbox_world[:3,i] for i in [4,5,6,7]],
        [bbox_world[:3,i] for i in [0,1,5,4]],
        [bbox_world[:3,i] for i in [2,3,7,6]],
        [bbox_world[:3,i] for i in [1,2,6,5]],
        [bbox_world[:3,i] for i in [4,7,3,0]],
    ]
    ax.add_collection3d(Poly3DCollection(verts, facecolors='cyan', linewidths=1, edgecolors='k', alpha=0.25))

    # 画相机
    ax.scatter(cam_center[0], cam_center[1], cam_center[2], c='r', marker='o')
    for i in range(1,5):
        ax.plot([cam_center[0], cam_frustum[i,0]],
                [cam_center[1], cam_frustum[i,1]],
                [cam_center[2], cam_frustum[i,2]], 'r-')
    ax.plot_trisurf(cam_frustum[1:,0], cam_frustum[1:,1], cam_frustum[1:,2], alpha=0.2, color='red')

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.view_init(elev=30, azim=60)
    plt.show()

# ========== 示例运行 ==========
if __name__ == "__main__":
    # 假设路径
    cam = 0
    intrinsics_file = "intrinsics/0.txt"
    extrinsics_file = "extrinsics/0.txt"

    K = load_intrinsics_txt(intrinsics_file)
    T_cam_to_world = load_extrinsics_txt(extrinsics_file)

    # 你的 obj_to_world (通常来自标注 JSON)
    T_obj_to_world = np.array([
        [-0.9834028955500602,  0.03272949521331979, -0.17845874920227073, 3421664.967727904],
        [ 0.17826844087012478, -0.008666949380814864, -0.983943721448623,   378828.16119695076],
        [-0.03375067426722838, -0.9994266677107989,   0.0026884670345344083,        5.349477577049384],
        [0.0, 0.0, 0.0, 1.0]
    ])

    box_size = (4.61556819538297, 1.946660239961535, 1.5889909214014342)

    plot_scene(T_obj_to_world, T_cam_to_world, box_size)